In [ ]:
import sys
sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')
from timm import create_model

In [ ]:
from fastai.vision.all import *

In [ ]:
def petfinder_rmse(input,target):
    return 100*torch.sqrt(F.mse_loss(F.sigmoid(input.flatten()), target))

In [ ]:
path = '../input/swin-transformer5foldmodels/models'
learn1 = load_learner('../input/swin-transformer5foldmodels/model_fold_2.pkl')

In [ ]:
dataset_path='../input/petfinder-pawpularity-score'
test_df = pd.read_csv(dataset_path+'/test.csv')
test_df.head()

In [ ]:
train_df = pd.read_csv(dataset_path+'/train.csv')
train_df['path'] = train_df['Id'].map(lambda x:str(dataset_path+'/train/'+x)+'.jpg')
train_df = train_df.drop(columns=['Id'])
train_df = train_df.sample(frac=1).reset_index(drop=True) #shuffle dataframe
train_df.head()

In [ ]:
test_df['Pawpularity'] = [1]*len(test_df)
test_df['path'] = test_df['Id'].map(lambda x:str(dataset_path+'/test/'+x)+'.jpg')
test_df = test_df.drop(columns=['Id'])
train_df['norm_score'] = train_df['Pawpularity']/100

In [ ]:
dls = ImageDataLoaders.from_df(train_df, #pass in train DataFrame
                               valid_pct=0.2, #80-20 train-validation random split
                               seed=999, #seed
                               fn_col='path', #filename/path is in the second column of the DataFrame
                               label_col='norm_score', #label is in the first column of the DataFrame
                               y_block=RegressionBlock, #The type of target
                               bs=32, #pass in batch size
                               num_workers=8,
                               item_tfms=Resize(224), #pass in item_tfms
                               batch_tfms=setup_aug_tfms([Brightness(), Contrast(), Hue(), Saturation()])) 
test_dl = dls.test_dl(test_df)

In [ ]:
test_dl.show_batch()

In [ ]:
preds, _ = learn1.tta(dl=test_dl, n=5, beta=0)

In [ ]:
sample_df = pd.read_csv(dataset_path+'/sample_submission.csv')
sample_df['Pawpularity'] = preds.float().numpy()*100
sample_df.to_csv('submission.csv',index=False)